In [0]:
dbutils.library.restartPython() 

In [0]:
!pip install prince
import prince
from scipy import stats
import os
!pip install numpy==1.24.0 --upgrade
import numpy as np
!pip install ydata_profiling
#import numpy as np
from ydata_profiling import ProfileReport
import pandas as pd



In [0]:
 
  ahs2023=pd.read_csv('/Workspace/Users/eashan@gmail.com/ahs2023_cleaned.csv')

In [0]:
ahs2023.head(3)
#type(ahs2023.dtypes)

In [0]:
for col in ahs2023:
    if ahs2023[col].dtype in ['object','string']:
       ahs2023[col] = ahs2023[col].astype('category')
ahs2023.dtypes

In [0]:
profile = ProfileReport(
        ahs2023, 
        title = '2023 American Housing Survey',
        html = {'style':{'full_width':True}},
        minimal = False        
        )
profile.to_notebook_iframe()


In [0]:
ahs2023['TENURE'].value_counts() 

In [0]:
ahs2023.groupby('TENURE').agg({'HINCP':['mean','count']},)

In [0]:
stats.f_oneway(
        ahs2023.query("TENURE=='Owned or being bought by someone in your household'")['HINCP'],
        ahs2023.query("TENURE=='Rented'")['HINCP'] ,
        ahs2023.query("TENURE=='Occupied without payment of rent'")['HINCP'])

In [0]:
ahs2023_for_pearsons_r = ahs2023[['MARKETVAL','YRBUILT']].dropna()
ahs2023_for_pearsons_r.corr()
#

In [0]:
stats.pearsonr(ahs2023_for_pearsons_r['MARKETVAL'],ahs2023_for_pearsons_r['YRBUILT'])

In [0]:
brokeness = ahs2023[['FNDCRUMB','PAINTPEEL','ROOFHOLE','ROOFSAG','ROOFSHIN','WALLCRACK','WALLSIDE','WALLSLOPE','WINBOARD','WINBROKE','LEAKI','MOLDBATH']].dropna()
brokeness.describe()
#brokeness['FUSEBLOW'].value_counts()
#brokeness['SEWBREAK'].value_counts()
#brokeness['ROACH'].value_counts()
#brokeness['RODENT'].value_counts()
#brokeness['NOWIRE'].value_counts()

In [0]:
#didnt want to sink too much time into debugging the below
brokeness.dropna(inplace=True)
brokeness

In [0]:
mca = prince.MCA(n_components=2)
MCA=mca.fit(brokeness.replace([np.inf, -np.inf], np.nan).dropna())

In [0]:
MCA.column_coordinates(brokeness).sort_values(by=0,ascending=False).values[0]

In [0]:
broken_index = MCA.row_coordinates(brokeness).sort_values(by=0,ascending=False)
broken_index = broken_index.rename({0:'house_condition',
                                   1:'other_index'},axis=1)

In [0]:
ahs_broken =  pd.merge(ahs2023,broken_index,how='outer', validate='one_to_one', indicator='matched', left_index=True, right_index=True
) 

In [0]:
ahs2023

In [0]:
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline


In [0]:
fig = sns.histplot(data=ahs_broken['HINCP'])
import matplotlib.pyplot as plt; plt.show()

In [0]:
fig, ax = plt.subplots(figsize=(10, 4))

sns.histplot(ahs_broken['HINCP'], bins=100, ax=ax)
ax.set_xscale('log')
plt.xlabel('Household Income')
plt.title('Distribution of Household Income')
plt.show()

In [0]:
ahs_tenure = ahs2023.groupby('TENURE').size().reset_index()
ahs_tenure = ahs_tenure.rename({0:'count'},axis=1) 
sns.barplot(data=ahs_tenure
            ,x=ahs_tenure['TENURE'],y=ahs_tenure['count'])
plt.show()

In [0]:
ahs_broken['house_condition']
sns.scatterplot(x=
                'house_condition',y='MARKETVAL', data=ahs_broken)
plt.show()

In [0]:
ahs_time = ahs_broken.groupby('YRBUILT').agg({'MARKETVAL':'mean','house_condition':'mean'})
sns.lineplot(x='YRBUILT',y='MARKETVAL',data=ahs_time)

In [0]:
ahs_facet = ahs_broken.groupby(['TENURE','DIVISION']).agg({'HINCP':'mean'}).reset_index()

g = sns.FacetGrid(ahs_facet, col = 'DIVISION', hue = 
'DIVISION',col_wrap=3)
g.map(sns.barplot, 'TENURE', 'HINCP')